In [2]:

#import neccessary packages

import numpy as np
import pandas as pd
import subprocess
import sys
import re
import os
import warnings


import pickle



from scipy.optimize import least_squares
from IPython.core.display import display
from IPython.core.debugger import set_trace
pd.set_option('display.max_columns', None,'display.max_rows', None)

warnings.filterwarnings('ignore')

from importlib import reload


In [3]:
# Import the neccessary modules

Module_path = '../Modules'  #specify path to modules folder, change as needed for environment


sys.path.insert(0,Module_path)


import base_functions as baf
import flux_opt as fopt


reload(fopt)
reload(baf)




<module 'base_functions' from '/Users/king339/Projects/Predictive_Phenomics/Code/Bolt/02_Fungal_model_analysis/../Modules/base_functions.py'>

## IMPORT A MODEL

This model can be sourced from the github repo Fungal-Metabolic-Model

In [7]:


model_file_dir = '../../Models/Fungal-Metabolic-Model'  #specify path to model file



file_name = 'Fungal-metabolic-model.pkl'



#Load the model file
S_active,active_reactions, metabolites, Keq= baf.load_model(model_file)
S, Keq, f_log_counts, vcount_upper_bound = baf.numpy_model_preprocessing(model_file_dir + '/' + file_name)


## OPTIMIZE GROWTH

##### Set flux objective coefficients

In [9]:
#for this example will simply maximize flux through all reactions equally
obj_rxn_list = ['protein_syn', 'rna_syn','dna_syn']



# Construct a dictionary mapping reaction names to their index in the stochiometric matrix
rxn_names = list(S_active.index)
n_react = len(rxn_names)
rxn_S_idx = np.arange(0,len(rxn_names))
rxn_name_to_idx = dict(zip(rxn_names,rxn_S_idx))

obj_rxn_idx = [rxn_name_to_idx[x] for x in obj_rxn_list]

obj_coefs = np.zeros(n_react)
obj_coefs[obj_rxn_idx] = 1

In [10]:
y_sol, alpha_sol, n_sol = fopt.flux_ent_opt(obj_coefs,vcount_upper_bound,f_log_counts,S,Keq)

Ipopt 3.14.4: max_iter=10000
max_cpu_time=800000
tol=1e-07
acceptable_tol=1e-06
linear_solver=ma57
hsllib=/opt/coinhsl/lib/libcoinhsl.dylib


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.4, running with linear solver ma57.

Number of nonzeros in equality constraint Jacobian...:     8037
Number of nonzeros in inequality constraint Jacobian.:     2422
Number of nonzeros in Lagrangian Hessian.............:      494

Input Error: Incorrect refinement scheme.
Total number of variables............................:     1235
                     variables with only lower bounds:        0
                variables with lower and uppe

### SAVE THE SOLUTION

In [11]:
#### Save the solution

save_file = 'max_growth_solution.pkl'


baf.save_model_solution(save_file,y_sol,alpha_sol,n_sol,S,Keq,f_log_counts,vcount_upper_bound,obj_coefs)
